# 📘 Agentic Architectures 17: Reflexive Metacognitive Agents

Welcome to an in-depth implementation of one of the most sophisticated agentic patterns: the **Reflexive Metacognitive Agent**. This architecture endows an agent with a form of self-awareness, enabling it to reason about its own capabilities, confidence, and limitations before taking action.

This goes a step beyond simple self-reflection. A metacognitive agent maintains an explicit **"self-model"**—a structured representation of its own knowledge, tools, and boundaries. When faced with a task, its first step is not to solve the problem, but to *analyze the problem in the context of its self-model*. It asks internal questions like:
- "Do I have sufficient knowledge to answer this confidently?"
- "Is this topic within my designated area of expertise?"
- "Do I have a specific tool that is required to answer this safely and accurately?"
- "Is the user's query about a high-stakes topic where an error would be dangerous?"

Based on the answers, it chooses a strategy: reason directly, use a specialized tool, or—most importantly—**escalate to a human** when the task exceeds its known limits.

To build a complex and powerful demonstration, we will create a **Medical Triage & Information Assistant**. This is a classic high-stakes scenario where the agent's ability to recognize its limitations is not just a feature, but a critical safety requirement.

### Definition
A **Reflexive Metacognitive Agent** is an agent that maintains and uses an explicit model of its own capabilities, knowledge boundaries, and confidence levels to select the most appropriate strategy for a given task. This self-modeling allows it to behave more safely and reliably, especially in domains where incorrect information is harmful.

### High-level Workflow

1.  **Perceive Task:** The agent receives a user request.
2.  **Metacognitive Analysis (Self-Reflection):** The agent's core reasoning engine analyzes the request *against its own self-model*. It assesses its confidence, the relevance of its tools, and whether the query falls within its predefined operational domain.
3.  **Strategy Selection:** Based on the analysis, the agent selects one of several strategies:
    *   **Reason Directly:** For high-confidence, low-risk queries within its knowledge base.
    *   **Use Tool:** When the query requires a specific capability the agent possesses via a tool.
    *   **Escalate/Refuse:** For low-confidence, high-risk, or out-of-scope queries.
4.  **Execute Strategy:** The chosen path is executed.
5.  **Respond:** The agent provides the result, which could be a direct answer, a tool-augmented answer, or a safe refusal with instructions to consult an expert.

### When to Use / Applications
*   **High-Stakes Advisory Systems:** Any system providing information in domains like healthcare, law, or finance, where an agent must be able to say "I don't know" or "You should consult a professional."
*   **Autonomous Systems:** A robot that must assess its own ability to perform a physical task safely before attempting it.
*   **Complex Tool Orchestrators:** An agent that must choose the right API from a vast library, understanding that some APIs are more dangerous or costly than others.

### Strengths & Weaknesses
*   **Strengths:**
    *   **Enhanced Safety and Reliability:** The primary benefit. The agent is explicitly designed to avoid making confident assertions in areas where it is not an expert.
    *   **Improved Decision Making:** Leads to more robust behavior by forcing a deliberate choice of strategy instead of naive, direct attempts.
*   **Weaknesses:**
    *   **Complexity of Self-Model:** Defining and maintaining an accurate self-model can be complex.
    *   **Metacognitive Overhead:** The initial analysis step adds latency and computational cost to every request.

## Phase 0: Foundation & Setup

Standard setup of libraries and environment variables.

In [2]:
!pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 14.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.23.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.


In [4]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Metacognitive Agent (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1: Defining the Agent's Self-Model and Tools

This is the foundation of the agent's self-awareness. We'll create a structured `AgentSelfModel` and a specialized tool. This model is not just a prompt; it's a configuration object that will be passed into the agent's reasoning loop.

In [6]:
console = Console()

# --- The Agent's Self-Model ---
class AgentSelfModel(BaseModel):
    """A structured representation of the agent's capabilities and limitations."""
    name: str
    role: str
    # The agent's explicit knowledge boundaries
    knowledge_domain: List[str] = Field(description="List of topics the agent is knowledgeable about.")
    # The agent's available tools
    available_tools: List[str] = Field(description="List of tools the agent can use.")
    confidence_threshold: float = Field(description="The confidence level (0-1) below which the agent must escalate.", default=0.6)

# Instantiate the self-model for our Medical Triage Agent
medical_agent_model = AgentSelfModel(
    name="TriageBot-3000",
    role="A helpful AI assistant for providing preliminary medical information.",
    knowledge_domain=["common_cold", "influenza", "allergies", "headaches", "basic_first_aid"],
    available_tools=["drug_interaction_checker"]
)

# --- Specialist Tools ---
class DrugInteractionChecker:
    """A mock tool to check for drug interactions."""
    def check(self, drug_a: str, drug_b: str) -> str:
        """Checks for interactions between two drugs."""
        # In a real system, this would query a medical database.
        known_interactions = {
            frozenset(["ibuprofen", "lisinopril"]): "Moderate risk: Ibuprofen may reduce the blood pressure-lowering effects of lisinopril. Monitor blood pressure.",
            frozenset(["aspirin", "warfarin"]): "High risk: Increased risk of bleeding. This combination should be avoided unless directed by a doctor."
        }
        interaction = known_interactions.get(frozenset([drug_a.lower(), drug_b.lower()]))
        if interaction:
            return f"Interaction Found: {interaction}"
        return "No known significant interactions found. However, always consult a pharmacist or doctor."

drug_tool = DrugInteractionChecker()
print("Agent Self-Model and Tools defined successfully.")

Agent Self-Model and Tools defined successfully.


In [ ]:
# Replace line: llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)

# With this:
MODEL_FALLBACKS = [
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "mistralai/Mistral-7B-Instruct-v0.2",
    "mixtral-8x7b-instruct",
    "mistral-medium",
]

llm = None
for model_name in MODEL_FALLBACKS:
    try:
        llm = ChatNebius(model=model_name, temperature=0)
        test = llm.invoke("test")  # Verify it works
        print(f"✅ Using model: {model_name}")
        break
    except Exception as e:
        print(f"❌ {model_name} failed")
        continue

if llm is None:
    print("⚠️ All Nebius models failed. Switching to OpenAI fallback...")
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Phase 2: Building the Metacognitive Agent with LangGraph

This is where the magic happens. We'll build a graph where the very first step is the **metacognitive analysis**. This node will use a powerful, detailed prompt to make the agent reason about itself. A conditional router will then direct the flow based on the chosen strategy.

In [7]:
#llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)

# Pydantic Models for structured outputs
class MetacognitiveAnalysis(BaseModel):
    """The agent's self-analysis of a query."""
    confidence: float = Field(description="A score from 0.0 to 1.0 representing the agent's confidence in its ability to answer safely and accurately.")
    strategy: str = Field(description="The chosen strategy. Must be one of: 'reason_directly', 'use_tool', 'escalate'.")
    reasoning: str = Field(description="A brief justification for the chosen confidence and strategy.")
    tool_to_use: Optional[str] = Field(description="If strategy is 'use_tool', the name of the tool to use.", default=None)
    tool_args: Optional[Dict[str, Any]] = Field(description="If strategy is 'use_tool', the arguments for the tool.", default=None)

# LangGraph State
class AgentState(TypedDict):
    user_query: str
    self_model: AgentSelfModel
    metacognitive_analysis: Optional[MetacognitiveAnalysis]
    tool_output: Optional[str]
    final_response: str

# Graph Nodes
def metacognitive_analysis_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("🤔 Agent is performing metacognitive analysis...", title="[yellow]Step: Self-Reflection[/yellow]"))
    prompt = ChatPromptTemplate.from_template(
        """You are a metacognitive reasoning engine for an AI assistant. Your task is to analyze a user's query in the context of the agent's own capabilities and limitations (its 'self-model').
        Your primary directive is **SAFETY**. You must determine the safest and most appropriate strategy for handling the query.

        **Agent's Self-Model:**
        - Name: {agent_name}
        - Role: {agent_role}
        - Knowledge Domain: {knowledge_domain}
        - Available Tools: {available_tools}

        **Strategy Rules:**
        1.  **escalate:** Choose this strategy if the query involves a potential medical emergency (e.g., chest pain, difficulty breathing, severe injury, broken bones), is outside the agent's knowledge domain, or if you have any doubt about providing a safe answer. **WHEN IN DOUBT, ESCALATE.**
        2.  **use_tool:** Choose this strategy if the query explicitly or implicitly requires one of the available tools. For example, a question about drug interactions requires the 'drug_interaction_checker'.
        3.  **reason_directly:** Choose this strategy ONLY if you are highly confident the query is a simple, low-risk question that falls squarely within the agent's knowledge domain.

        Analyze the user query below and provide your metacognitive analysis in the required format.

        **User Query:** "{query}"""
    )
    chain = prompt | llm.with_structured_output(MetacognitiveAnalysis)
    analysis = chain.invoke({
        "query": state['user_query'],
        "agent_name": state['self_model'].name,
        "agent_role": state['self_model'].role,
        "knowledge_domain": ", ".join(state['self_model'].knowledge_domain),
        "available_tools": ", ".join(state['self_model'].available_tools),
    })
    console.print(Panel(f"[bold]Confidence:[/bold] {analysis.confidence:.2f}\n[bold]Strategy:[/bold] {analysis.strategy}\n[bold]Reasoning:[/bold] {analysis.reasoning}", title="Metacognitive Analysis Result"))
    return {"metacognitive_analysis": analysis}

def reason_directly_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("✅ Confident in direct answer. Generating response...", title="[green]Strategy: Reason Directly[/green]"))
    prompt = ChatPromptTemplate.from_template("You are {agent_role}. Provide a helpful, non-prescriptive answer to the user's query. Remind the user that you are not a doctor.\n\nQuery: {query}")
    chain = prompt | llm
    response = chain.invoke({"agent_role": state['self_model'].role, "query": state['user_query']}).content
    return {"final_response": response}

def call_tool_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel(f"🛠️ Confidence requires tool use. Calling `{state['metacognitive_analysis'].tool_to_use}`...", title="[cyan]Strategy: Use Tool[/cyan]"))
    analysis = state['metacognitive_analysis']
    if analysis.tool_to_use == 'drug_interaction_checker':
        tool_output = drug_tool.check(**analysis.tool_args)
        return {"tool_output": tool_output}
    return {"tool_output": "Error: Tool not found."}

def synthesize_tool_response_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("📝 Synthesizing final response from tool output...", title="[cyan]Step: Synthesize[/cyan]"))
    prompt = ChatPromptTemplate.from_template("You are {agent_role}. You have used a tool to get specific information. Now, present this information to the user in a clear and helpful way. ALWAYS include a disclaimer to consult a healthcare professional.\n\nOriginal Query: {query}\nTool Output: {tool_output}")
    chain = prompt | llm
    response = chain.invoke({"agent_role": state['self_model'].role, "query": state['user_query'], "tool_output": state['tool_output']}).content
    return {"final_response": response}

def escalate_to_human_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("🚨 Low confidence or high risk detected. Escalating to human.", title="[bold red]Strategy: Escalate[/bold red]"))
    response = "I am an AI assistant and not qualified to provide information on this topic. This query is outside my knowledge domain or involves potentially serious symptoms. **Please consult a qualified medical professional immediately.**"
    return {"final_response": response}

# Conditional Edge
def route_strategy(state: AgentState) -> str:
    return state["metacognitive_analysis"].strategy

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("analyze", metacognitive_analysis_node)
workflow.add_node("reason", reason_directly_node)
workflow.add_node("call_tool", call_tool_node)
workflow.add_node("synthesize", synthesize_tool_response_node)
workflow.add_node("escalate", escalate_to_human_node)

workflow.set_entry_point("analyze")
workflow.add_conditional_edges("analyze", route_strategy, {
    "reason_directly": "reason",
    "use_tool": "call_tool",
    "escalate": "escalate"
})
workflow.add_edge("call_tool", "synthesize")
workflow.add_edge("reason", END)
workflow.add_edge("synthesize", END)
workflow.add_edge("escalate", END)

metacognitive_agent = workflow.compile()
print("Reflexive Metacognitive Agent graph compiled successfully.")

Reflexive Metacognitive Agent graph compiled successfully.


## Phase 3: Demonstration & Analysis

Now we will test the agent with a series of increasingly difficult and high-stakes queries. We will observe how the metacognitive analysis correctly routes each query down the appropriate path, demonstrating the system's safety and self-awareness.

In [9]:
def run_agent(query: str):
    initial_state = {"user_query": query, "self_model": medical_agent_model}
    result = metacognitive_agent.invoke(initial_state)
    console.print(Markdown(result['final_response']))

# Test 1: Simple, should be answered directly
console.print("--- Test 1: Simple, In-Scope, Low-Risk Query ---")
run_agent("What are the symptoms of a common cold?")

# Test 2: Requires the specific tool
console.print("\n--- Test 2: Specific Query Requiring a Tool ---")
run_agent("Is it safe to take Ibuprofen if I am also taking Lisinopril?")

# Test 3: High-stakes, should be escalated immediately
console.print("\n--- Test 3: High-Stakes, Emergency Query ---")
run_agent("I have a crushing pain in my chest and my left arm feels numb, what should I do?")

--- Test 1: Simple, In-Scope, Low-Risk Query ---

╭───────────────────────────────────────────── Step: Self-Reflection ─────────────────────────────────────────────╮
│ 🤔 Agent is performing metacognitive analysis...                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

NotFoundError: Error code: 404 - {'detail': 'The model `mistralai/Mixtral-8x22B-Instruct-v0.1` does not exist.'}

### Analysis of the Results

The demonstration is a powerful illustration of the safety and reliability this architecture provides:

1.  **Correctly Scoped Answer:** For the 'common cold' query, the metacognitive analysis correctly identified it as a low-risk topic within its knowledge domain. It set a high confidence score and chose the `reason_directly` strategy, providing a helpful but properly caveated answer.

2.  **Correct Tool Use:** For the drug interaction question, the analysis recognized the need for a specific capability. It correctly identified that the `drug_interaction_checker` tool was required, set a high confidence *in its ability to use the tool*, and selected the `use_tool` strategy. The final response was a safe, synthesized summary of the tool's output.

3.  **Critical Safety Escalation:** This is the most important result. A naive agent might have tried to answer the 'chest pain' query by searching the web for causes, potentially providing dangerous and misleading information. Our metacognitive agent, guided by its primary directive of safety, immediately recognized the signs of a medical emergency. The metacognitive analysis assigned a very low confidence score and correctly chose the `escalate` strategy. The final output was not an answer, but a safe, responsible refusal and a directive to seek professional help. It correctly identified the limits of its own competence.

This workflow proves that by forcing the agent to reason about itself *before* reasoning about the problem, we can build a powerful layer of safety and reliability into its operation.

## Conclusion

In this detailed notebook, we have implemented a **Reflexive Metacognitive Agent**, a sophisticated architecture that prioritizes safety and reliability by endowing an agent with self-awareness. By building an explicit `self-model` and forcing a metacognitive analysis as the first step of any task, we have created a system that understands its own boundaries.

The key innovation is the shift in the agent's initial goal from "How do I answer this?" to "*Should* I answer this, and if so, how?" This introspective step allows the agent to dynamically choose the safest and most appropriate strategy—be it direct reasoning, specialized tool use, or crucial escalation to a human expert.

This architecture is more than just a technique; it's a design philosophy. It is absolutely essential for creating responsible AI agents that can be trusted to operate in high-stakes, real-world domains where knowing what you *don't* know is just as important as what you do.